In [16]:
from bs4 import BeautifulSoup
import requests
import json

In [66]:
import pymysql

# 資料庫參數設定
db_settings = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "[password]",
    "db": "crawledData",
    "charset": ""
}

In [14]:
# 避免最後一個字是頓號
def comma_filter(s):
    if(s[-1] == "、"):
        s = s[:-1]
    return s

In [21]:
# 起始爬取頁次（從第二頁開始爬）
page = 2
# 最終爬取頁次
max_page = 4
# 存放貼文連結
links = []

require_name = {"workExp": "工作經歷", "edu": "學歷要求", "major": "科系要求", "language": "語文條件", "specialty": "擅長工具", 
"skill": "工作技能", "certificate": "具備證照", "driverLicense": "具備駕照", "other": "其他條件"}
contact_name = {"hrName": "聯絡人", "email": "E-mail", "phone": "電洽", "other": "其他",
"reply": "應徵回覆"}

In [35]:
# 爬取每篇貼文連結

while(page < max_page):
    url = "https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&isnew=14&kwop=7&keyword=實習%20intern&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=7&asc=0&sr=99&rostatus=1024&page=" + str(page) + "&mode=s&langFlag=0&langStatus=0&recommendJob=1&hotJob=1"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    posts_links = soup.find_all(class_ = "js-job-link")
    page += 1
    # 儲存該頁的每則貼文連結
    for link in posts_links:
        links.append(link['href'])

In [ ]:
for link in links:
    # 實際請求 api
    ind_start = link.find("job/")
    ind_end = link.find("?")
    ajax_url = "https:" + link[:ind_start + 4] + "ajax/content/" + link[ind_start + 4:ind_end]

    # 需註明來源網址（Referer 參數）才能取得貼文內容
    headers = {
        "Referer": ("https:" + link),
    }
    r_content = requests.get(url = ajax_url, headers = headers)
    # data 為送 request 回傳的 json 格式內容，即是貼文內容
    data = json.loads(r_content.text)
    
    title = data["data"]["header"]["jobName"]
    cp_name = data["data"]["header"]["custName"]
    job_desc = data["data"]["jobDetail"]["jobDescription"]
    complete_link = "https:" + link[:ind_end]
    
    # 條件要求
    requirement = ""
    for d in data["data"]["condition"]:
        # 不爬此欄位
        if(d == "acceptRole" or (d == "localLanguage")):
            continue
        # 沒有 value 的 key
        if(data["data"]["condition"][d] == [] or (data["data"]["condition"][d] == '')):
            continue
        
        # 條件要求欄位名
        requirement += require_name[d] + ": "
        # 其他條件
        if(d == "other"):
            requirement += "\n"
        # 擅長工具
        if(d == "specialty"):
            for j in data["data"]["condition"]["specialty"]:
                requirement += (j["description"] + "、")
        # 科系要求
        elif(d == "major"):
            for j in data["data"]["condition"]["major"]:
                requirement += (j + "、") 
        # 語言條件
        elif(d == "language"):
            for k in data["data"]["condition"]["language"]:
                requirement += (k['language'] + ": " + k["ability"])
        # 具備駕照
        elif(d == "driverLicense"):
            for k in data["data"]["condition"][d]:
                requirement += (k + "、")
        # 工作技能
        elif(d == "skill"):
            for k in data["data"]["condition"][d]:
                requirement += (k["description"] + "、")
        else:
            requirement += data["data"]["condition"][d]
        requirement = comma_filter(requirement)
        requirement += "\n"
        
    # 待遇福利
    benefits = data["data"]["jobDetail"]["salary"]
    if(data["data"]["welfare"]["legalTag"] != []):
        benefits += "\n法定項目: "
        for b in data["data"]["welfare"]["legalTag"]:
            benefits += (b + "、")
        benefits = comma_filter(benefits)
    if(data["data"]["welfare"]["welfare"] != ''):
        benefits += ("\n其他福利:\n" + data["data"]["welfare"]["welfare"])
    # 地址
    location = (data["data"]["jobDetail"]["addressRegion"] + data["data"]["jobDetail"]["addressDetail"])
    # 更新日期
    date = data["data"]["header"]["appearDate"]
    # 其他(聯絡資訊)
    other = "聯絡資訊: "
    for c in data["data"]["contact"]:
        if(c == "suggestExam" or (c == "visit")):
            continue
        if(data["data"]["contact"][c] == '' or data["data"]["contact"][c] == []):
            continue
        if(c == "other"):
            other += ("\n" + contact_name[c] + ": \n")
        elif(c == "phone"):
            other += ("\n" + contact_name[c] + ": ")
            other += data["data"]["contact"][c][0]
            continue
        else:
            other += ("\n" + contact_name[c] + ": ")
        other += data["data"]["contact"][c]
    
    # 存入 DB
    try:
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            insert = "INSERT INTO 104_content(link, title, cp_name, job_desc, requirement, benefits, location, other) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(insert, (complete_link ,title, cp_name, job_desc, requirement, benefits, location, other))
            conn.commit()
    except Exception as ex:
        print(ex)
        continue